In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain.vectorstores import Chroma        


In [ ]:

# read .key file to get openai api key
with open('openai_api_key.key', 'r') as f:
    openai_api_key = f.read().strip()

embeddings = OpenAIEmbeddings(
        model='text-embedding-ada-002', openai_api_key=openai_api_key)



In [ ]:
# from pickle import the text list (all the article from code civil)
import pickle

with open('text_list.pickle', 'rb') as f:
    text_list = pickle.load(f)


In [ ]:
vectorstore = Chroma.from_documents(
            text_list, embeddings, collection_name="my_collection", persist_directory="../vectorstore")

vectorstore.persist()